In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
print(os.listdir("./input"))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

['unigram_freq.csv']


In [3]:
df = pd.read_csv('./input/unigram_freq.csv')
print(df.shape)
df.dropna(axis=0,how='any') # 결측값(NaN)이 있는 행 삭제
print(df.shape)

(333333, 2)
(333333, 2)


In [4]:
lines = [x for x in df['word'] if type(x) == type('a')]
print("Line Count:", len(lines))
print(lines[:4])

Line Count: 333331
['the', 'of', 'and', 'to']


In [5]:
import re
def process(sent):
    sent= sent.lower()
    sent= re.sub(r'[^0-9a-zA-Z]', '', sent)
    sent= sent.replace('\n','')
    return sent

In [6]:
lines = [process(x) for x in lines]
temp = []
for line in lines:
    temp+= [ x for x in line.split() ]
lines = list(set(temp))
print("\n".join(lines[:4]))
print("Number of items:",len(lines))

prouect
nzoss
nonprime
golem
Number of items: 333331


In [7]:
char_set = list(" abcdefghijklmnopqrstuvwxyz0123456789")
char2int = { char_set[x]:x for x in range(len(char_set)) }
int2char = { char2int[x]:x for x in char_set }
print(char2int)
print(int2char)


{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36}
{0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '0', 28: '1', 29: '2', 30: '3', 31: '4', 32: '5', 33: '6', 34: '7', 35: '8', 36: '9'}


In [8]:
count = len(char_set)
codes = ["\t","\n",'#']
for i in range(len(codes)):
    code = codes[i]
    char2int[code]=count
    int2char[count]=code
    count+=1
print(char2int)
print(int2char)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, '\t': 37, '\n': 38, '#': 39}
{0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '0', 28: '1', 29: '2', 30: '3', 31: '4', 32: '5', 33: '6', 34: '7', 35: '8', 36: '9', 37: '\t', 38: '\n', 39: '#'}


In [26]:
import random
#thresh - 0 to 1  얼만큼 바뀌게 할건지(오타가 얼마나 심한지)
def gen_gibberish(line,thresh=0.2):
    times = int(random.randrange(1,len(line)) * thresh)
    '''
    Types of replacement: 어떻게 오타가 발생하는지
        1.Delete random character.
        2.Add random character.
        3.Replace a character.
        4.Combination?
    '''
    while times!=0:
        times-=1
        val = random.randrange(0,10)
        if val <= 5:
            val = random.randrange(0,10)
            index = random.randrange(2,len(line))
            if val <=3 :
                line = line[:index]+line[index+1:]
            else:
                insert_index = random.randrange(0, len(char_set))
                line = line[:index] + char_set[insert_index] + line[index:]
            
        else:
            index = random.randrange(0, len(char_set))
            replace_index = random.randrange(2,len(line))
            line = line[:replace_index] + char_set[index] + line[replace_index+1:]
    return line

sample = lines[5]
gib = gen_gibberish(sample)
print("Original:", sample)
print("Gibberish:", gib)




Original: selina
Gibberish: selina


In [27]:
#create dataset 오타 데이터셋

input_texts = []
target_texts = []
REPEAT_FACTOR = 1
SKIP = int(len(lines)*0.65)

for line in lines[SKIP:]:
    if len(line) > 10:
        output_text = '\t' + line + '\n'
        for _ in range(REPEAT_FACTOR):
            input_text = gen_gibberish(line)
            input_texts.append(input_text)
            target_texts.append(output_text)
print("LEN OF SAMPLES:",len(input_texts))



LEN OF SAMPLES: 15135


In [28]:
max_enc_len = max([len(x) for x in input_texts])
max_dec_len = max([len(x) for x in target_texts])
print("Max Enc Len:", max_enc_len)
print("Max Dec Len:", max_dec_len)

Max Enc Len: 36
Max Dec Len: 38


In [29]:
num_samples = len(input_texts)
encoder_input_data = np.zeros((num_samples, max_enc_len, len(char_set)), dtype='float32')
decoder_input_data = np.zeros((num_samples, max_dec_len, len(char_set)+2), dtype='float32')
decoder_target_data = np.zeros((num_samples, max_dec_len, len(char_set)+2), dtype='float32')
print("CREATED ZERO VECTORS")

CREATED ZERO VECTORS


In [30]:
#filling in the enc,dec datas 원핫 벡터인거 같은데
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t, char2int[char]] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, char2int[char]] = 1
        if t > 0 :
            decoder_target_data[i, t-1, char2int[char]] = 1
print("COMPLETED...")
    # decoder_target은 한칸씩 당겨진거?

COMPLETED...


In [44]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)


In [45]:
batch_size = 128
epochs = 1000
latent_dim = 256

num_enc_tokens = len(char_set)
num_dec_tokens = len(char_set) + 2 # includes \n \t

encoder_inputs = Input(shape=(None, num_enc_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


In [46]:
decoder_inputs = Input(shape=(None,num_dec_tokens))
decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_ouputs,_,_ = decoder_lstm(decoder_inputs,initial_state = encoder_states)

decoder_dense = Dense(num_dec_tokens, activation='softmax')
decoder_ouputs = decoder_dense(decoder_ouputs)


model = Model([encoder_inputs,decoder_inputs],decoder_ouputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, None, 37)]   0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, None, 39)]   0           []                               
                                                                                                  
 lstm_5 (LSTM)                  [(None, 256),        301056      ['input_6[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

In [47]:
h=model.fit([encoder_input_data,decoder_input_data],decoder_target_data
         ,epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.2
         )
model.save('s2s.h5')

Epoch 1/1000
95/95 [==============================] - 14s 127ms/step - loss: 1.0637 - val_loss: 1.0253
Epoch 2/1000
95/95 [==============================] - 12s 122ms/step - loss: 1.0216 - val_loss: 1.0008
Epoch 3/1000
95/95 [==============================] - 12s 125ms/step - loss: 0.9940 - val_loss: 0.9852
Epoch 4/1000
95/95 [==============================] - 12s 124ms/step - loss: 0.9728 - val_loss: 0.9744
Epoch 5/1000
95/95 [==============================] - 12s 125ms/step - loss: 0.9575 - val_loss: 0.9448
Epoch 6/1000
95/95 [==============================] - 12s 128ms/step - loss: 0.9463 - val_loss: 0.9349
Epoch 7/1000
95/95 [==============================] - 12s 129ms/step - loss: 0.9375 - val_loss: 0.9288
Epoch 8/1000
95/95 [==============================] - 13s 135ms/step - loss: 0.9336 - val_loss: 0.9186
Epoch 9/1000
95/95 [==============================] - 13s 134ms/step - loss: 0.9283 - val_loss: 0.9177
Epoch 10/1000
95/95 [==============================] - 13s 137ms/step - l

KeyboardInterrupt: 